## Bagging 실습
- 집값 예측 프로그램 작성
- 배깅과 다른 모델들의 성능 비교

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
data = pd.read_csv("./data/kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


- id: 집 고유아이디
- date: 집이 팔린 날짜 
- price: 집 가격 (타겟변수)
- bedrooms: 주택 당 침실 개수
- bathrooms: 주택 당 화장실 개수
- floors: 전체 층 개수
- waterfront: 해변이 보이는지 (0, 1)
- condition: 집 청소상태 (1~5)
- grade: King County grading system 으로 인한 평점 (1~13)
- yr_built: 집이 지어진 년도
- yr_renovated: 집이 리모델링 된 년도
- zipcode: 우편번호
- lat: 위도
- long: 경도

In [7]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis=1)

In [9]:
data.head() # 범주형 변수 waterfront가 이진 분류로 변환되어 있으므로 수치형으로 변환 과정 생략

,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated
0,221900.0,3,1.00,1.0,0,3,7,1955,0
1,538000.0,3,2.25,2.0,0,3,7,1951,1991
2,180000.0,2,1.00,1.0,0,3,6,1933,0
3,604000.0,4,3.00,1.0,0,5,7,1965,0
4,510000.0,3,2.00,1.0,0,3,8,1987,0


In [11]:
feature_columns = list(data.columns.difference(['price']))
X = data[feature_columns]
y = data['price']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(15129, 8) (6484, 8) (15129,) (6484,)


#### 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models)

In [12]:
import statsmodels.api as sm # 다양한 통계 분석 시 사용
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [13]:
sm_train_x = sm.add_constant(train_x, has_constant='add') # Bias 추가
sm_model = sm.OLS(train_y, sm_train_x) # OLS 회귀 분석 모델
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Fri, 04 Feb 2022   Prob (F-statistic):               0.00
Time:                        21:33:55   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
sm_test_x = sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)
print('RMSE: {}'.format(sqrt(mean_squared_error(sm_model_predict, test_y))))
print(fitted_sm_model.params)

RMSE: 239703.0959110036
const           7.066913e+06
bathrooms       1.198305e+05
bedrooms        6.134945e+02
condition       1.660269e+04
floors          6.582927e+03
grade           1.974730e+05
waterfront      8.868726e+05
yr_built       -4.245187e+03
yr_renovated    1.329155e+01
dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [22]:
import random

In [32]:
# Bagging 결과와 비교
bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    # print(len(data_index))
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 복원 추출
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ]
    sm_train_y = train_y.iloc[random_data_index, ]
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()

    sm_test_x = sm.add_constant(test_x, has_constant='add')
    sm_model_predict = fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(sm_model_predict)


9574
9627
9575
9511
9597
9580
9542
9561
9546
9529


In [35]:
bagging_predict_result[0] # 집값 예측

735      5.624300e+05
2830     7.080162e+05
4106     1.118945e+06
16218    1.473368e+06
19964    6.974638e+05
             ...     
12606    5.987356e+05
14393    6.812996e+05
6899     3.289456e+05
85       9.148887e+05
21363    4.352112e+05
Length: 6484, dtype: float64

In [25]:
test_x.shape

(6484, 8)

In [36]:
bagging_predict = []
for lst2_index in range(test_x.shape[0]): # 6484
    temp_predict = [] # 반복문 안에서 결과값 저장
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [39]:
bagging_predict

[563209.493411783,
 712563.666029538,
 1116111.5776317634,
 1472347.3453183426,
 697174.3086934555,
 385549.316143237,
 786451.3101974729,
 486422.3952258746,
 497584.9477806796,
 539361.1114350834,
 643608.9255522105,
 411140.67165249505,
 267507.3344880114,
 278648.13785787893,
 339140.3547102296,
 1260203.8981736733,
 322981.6645409273,
 1032564.4770502892,
 260804.44461135223,
 603989.3792614517,
 391257.7512777115,
 1298898.7961937222,
 819841.9224128304,
 581515.2297511578,
 594372.3565767601,
 568642.6428913361,
 266379.19187134755,
 42295.37437973456,
 562520.9850807985,
 643269.456983505,
 565401.595731326,
 460054.33695542265,
 554145.9478544248,
 689316.1157775914,
 410521.67615921394,
 877402.2014283784,
 936698.6489324862,
 638018.3360618369,
 390485.71881820465,
 1077784.400966499,
 452883.23645712546,
 150351.44855516544,
 487569.38120998413,
 219420.5356922339,
 66789.83095857149,
 -42394.38583352221,
 250038.31136921947,
 282860.54724596476,
 362528.2560824009,
 722616

In [40]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 239631.34191469062


- 선형 호귀 모형에 평가 데이터 RMSE 구하기

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
regression_model = LinearRegression()
linear_model1 = regression_model.fit(train_x, train_y)
predict1 = linear_model1.predict(test_x)
print("RMSE : {}".format(sqrt(mean_squared_error(predict1, test_y))))

RMSE : 239804.29670858145


- Bagging을 이용하여 선형 회귀 모형에 평가(Sampling 10번)

In [43]:
from sklearn.ensemble import BaggingRegressor

In [47]:
bagging_model = BaggingRegressor(base_estimator=regression_model,
                                 n_estimators=30,
                                 verbose=1)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)
print("RMSE : {}".format(sqrt(mean_squared_error(predict2, test_y))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE : 239798.33466618566


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


- 의사결정나무모형에 적합시키기

In [48]:
from sklearn.tree import DecisionTreeRegressor

In [50]:
decision_tree_model = DecisionTreeRegressor()
tree_model1 = decision_tree_model.fit(train_x, train_y)
predict1 = tree_model1.predict(test_x)
print("RMSE : {}".format(sqrt(mean_squared_error(predict1, test_y))))

RMSE : 299416.917239664


In [51]:
bagging_decision_tree_model = BaggingRegressor(base_estimator=decision_tree_model,
                                               n_estimators=30,
                                               verbose=1)
tree_model2 = bagging_decision_tree_model.fit(train_x, train_y)
predict2 = tree_model2.predict(test_x)
print("RMSE : {}".format(sqrt(mean_squared_error(predict2, test_y))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE : 234807.7201047509


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
